In [1]:
import pandas as pd
from collections import defaultdict
from typing import DefaultDict
from pathlib import Path

import sys

SEP = " | "

In [2]:
pauq_train = pd.read_json("./en_iid_train.json")
tables = pd.read_json("./tables.json")

In [4]:
pauq_train

,id,db_id,source,type,query,question,sql,question_toks,query_toks,query_toks_no_values,masked_query
0,TS_0001,department_management,spider-train,train,SELECT count(*) FROM head WHERE age > 56;,How many heads of the departments are older th...,"{'from': {'table_units': [['table_unit', 1]], ...","[How, many, heads, of, the, departments, are, ...","[select, count, (, *, ), from, head, where, ag...","[select, count, (, *, ), from, head, where, ag...",SELECT count(*) FROM head WHERE age > NUMERIC_...
1,TS_0002,department_management,spider-train,train,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","{'from': {'table_units': [['table_unit', 1]], ...","[List, the, name, ,, born, state, and, age, of...","[select, name, ,, born_state, ,, age, from, he...","[select, name, ,, born_state, ,, age, from, he...","SELECT name , born_state , age FROM head ORDER..."
2,TS_0003,department_management,spider-train,train,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","{'from': {'table_units': [['table_unit', 0]], ...","[List, the, creation, year, ,, name, and, budg...","[select, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","SELECT creation , name , budget_in_billions FR..."
3,TS_0004,department_management,spider-train,train,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"{'from': {'table_units': [['table_unit', 0]], ...","[What, are, the, maximum, and, minimum, budget...","[select, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","SELECT max(budget_in_billions) , min(budget_in..."
4,TS_0005,department_management,spider-train,train,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"{'from': {'table_units': [['table_unit', 0]], ...","[What, is, the, average, number, of, employees...","[select, avg, (, num_employees, ), from, depar...","[select, avg, (, num_employees, ), from, depar...",SELECT avg(num_employees) FROM department WHER...
...,...,...,...,...,...,...,...,...,...,...,...
8795,L_0016,match_season,addition,train,SELECT DISTINCT(Season) FROM match_season WHER...,At what seasons was Paulo Petrov in substitute?,"{'from': {'table_units': [['table_unit', 2]], ...","[At, what, seasons, was, Paulo, Petrov, in, su...","[select, distinct, (, season, ), from, match_s...","[select, distinct, (, season, ), from, match_s...",SELECT DISTINCT(Season) FROM match_season WHER...
8796,L_0017,college_1,addition,train,"SELECT CRS_CREDIT, CRS_CODE FROM COURSE WHERE ...",Credits amount and code for course about the l...,"{'from': {'table_units': [['table_unit', 1]], ...","[Credits, amount, and, code, for, course, abou...","[select, crs_credit, ,, crs_code, from, course...","[select, crs_credit, ,, crs_code, from, course...","SELECT CRS_CREDIT, CRS_CODE FROM COURSE WHERE ..."
8797,L_0018,department_store,addition,train,"SELECT order_id , customer_id FROM customer_o...","Show, please, the order id, customer id for or...","{'from': {'table_units': [['table_unit', 8]], ...","[Show, ,, please, ,, the, order, id, ,, custom...","[select, order_id, ,, customer_id, from, custo...","[select, order_id, ,, customer_id, from, custo...","SELECT order_id , customer_id FROM customer_or..."
8798,L_0019,cre_Drama_Workshop_Groups,addition,train,SELECT COUNT(*) FROM Marketing_Regions WHERE M...,Marketing regions number for bad food.,"{'from': {'table_units': [['table_unit', 4]], ...","[Marketing, regions, number, for, bad, food, .]","[select, count, (, *, ), from, marketing_regio...","[select, count, (, *, ), from, marketing_regio...",SELECT COUNT(*) FROM Marketing_Regions WHERE M...


In [46]:
import numpy as np

d = []
for i in pauq_train['query_toks_no_values'].values:
    d.extend(i)
d = np.unique(np.array(d), return_counts=True)
dict(list(filter(lambda x : x[1] > 5, sorted(list(zip(d[0], d[1])), key=lambda x : x[1])))).keys()

dict_keys(['120', '180', '1995', '1996', '1999', '2003', '2006', '2017', '32', '50000', '8000', '85', '94107', 'account_name', 'accreditation_type', 'address_line_1', 'aid', 'airport_name', 'amount_payment', 'amount_piad', 'apartment_facilities', 'artists', 'attribute_data_type', 'attribute_definitions', 'attribute_value', 'bedroom_count', 'behavior_incident', 'birth_country', 'bname', 'builder', 'candidate_id', 'candidates', 'catalog_structure', 'claims_processing_stages', 'club_rank', 'coach', 'company_name', 'comptroller', 'country_id', 'country_name', 'course_name', 'culture_company', 'customer_last_name', 'date_assigned_to', 'date_became_customer', 'date_of_notes', 'date_order_placed', 'date_payment_made', 'dept_address', 'detention', 'document_status_code', 'documents_mailed', 'domain_publication', 'donator_name', 'dormid', 'dst_apid', 'emp_fname', 'famous_title', 'fault_log', 'fda_approved', 'film_actor', 'forename', 'gdp', 'gender_code', 'grant_amount', 'hall_of_fame', 'invoice

In [47]:
def wrap_quotes(x):
    """Wrap a string with double quotes."""
    return f'"{x}"'

def get_foreing_in_text(row: pd.Series) -> DefaultDict:
    """
    Extract foreign keys from a Pandas Series object and format them into a dictionary.

    Args:
    row (pd.Series): A Pandas Series containing database information, including foreign keys.

    Returns:
    defaultdict: A dictionary mapping table IDs to formatted foreign key information.
    """
    table_to_foreing_string = defaultdict(str)

    for fkey in row.foreign_keys:
        to_column_id, from_column_id = fkey
        (to_table_id, to_column_name), (from_table_id, from_column_name) = row.column_names[to_column_id], row.column_names[from_column_id]
        to_column_name, from_column_name = wrap_quotes(to_column_name), wrap_quotes(from_column_name)
        from_table_name, to_column_type = wrap_quotes(row.table_names[from_table_id]), row.column_types[to_column_id]
        
        table_to_foreing_string[to_table_id] += f"{to_column_name} {to_column_type} from {from_table_name} {from_column_name}, "

    return table_to_foreing_string

def get_primary_in_text(row: pd.Series) -> DefaultDict:
    """
    Extract primary keys from a Pandas Series object and format them into a dictionary.

    Args:
    row (pd.Series): A Pandas Series containing database information, including primary keys.

    Returns:
    defaultdict: A dictionary mapping table IDs to formatted primary key information.
    """
    table_to_primary_string = defaultdict(str)
    for pkey in row.primary_keys:
        # [1, 4]
        table_id, columns_name = row.column_names[pkey]
        columns_name = wrap_quotes(columns_name)
        
        if table_id not in table_to_primary_string:
            table_to_primary_string[table_id] = columns_name
        else: 
            table_to_primary_string[table_id] += f", {columns_name}"

    return table_to_primary_string
    

In [48]:
def get_schema(row: pd.Series) -> str:
    schemas = []
    foreing_keys = get_foreing_in_text(row)
    primary_keys = get_primary_in_text(row)
    
    for table_id, table_name in enumerate(row.table_names):
#         table_name = wrap_quotes(table_name)
        table_name = table_name


        columns_name_list = []
        for (col_table_id, name), type in zip(row.column_names, row.column_types):
            if col_table_id == table_id:
#                 columns_name_list.append(f"{wrap_quotes(name)} {type}")
                columns_name_list.append(f"{name}")

                
        columns_name = " , ".join(columns_name_list)

#         table_schema = f"{table_name} {columns_name} foreign_key: {foreing_keys[table_id]} primary_key: {primary_keys[table_id]}"
        table_schema = f"{table_name} : {columns_name}"

        schemas.append(table_schema)

    formatted_schema = f" {SEP} ".join(schemas)
    return formatted_schema

In [49]:
tables.loc[:, "query_schema"] = tables.apply(lambda x: get_schema(x), axis=1)

In [50]:
tables.loc[:, "query_schema"].sample(5).iloc[0]

'city : city id , official name , status , area km 2 , population , census ranking  |  farm : farm id , year , total horses , working horses , total cattle , oxen , bulls , cows , pigs , sheep and goats  |  farm competition : competition id , year , theme , host city id , hosts  |  competition record : competition id , farm id , rank'

In [51]:
def process_splits(data: pd.DataFrame, tables: pd.DataFrame, lang: str = "en") -> pd.DataFrame:
    assert lang in {"ru", "en"}, "There are only 2 available languages: 'ru' / 'en'"
    data = data.merge(tables[["db_id", "query_schema"]], on="db_id")[["query", "question", "query_schema"]]

    data.loc[:, "query"] = data.apply(lambda x: x["query"][:-1], axis=1)

    data.loc[:, "question"] = data.apply(lambda x: f'{x["question"]} {x["query_schema"]}', axis=1)
    data = data.drop("query_schema", axis=1)

    return data

In [52]:
prepared_pauq_train = process_splits(pauq_train, tables)

In [54]:
m = []
for i, j in prepared_pauq_train.to_records(index=False):
    tmp = (len(i.split()), len(j.split()))
    m.append(tmp)
m = np.array(m)

In [55]:
for i in m:
    if i[1] > 400:
        print(i)

[ 25 930]
[ 25 928]
[ 20 928]
[ 20 927]
[ 20 928]
[ 20 929]
[ 29 936]
[ 29 948]
[  9 922]
[  9 926]
[  9 926]
[  9 928]
[ 29 932]
[ 29 930]
[ 27 929]
[ 27 930]
[  8 923]
[  8 924]
[ 27 927]
[ 27 931]
[ 13 924]
[ 13 924]
[ 10 927]
[ 10 929]
[ 12 929]
[ 12 928]
[ 27 927]
[ 27 928]
[ 21 929]
[ 21 928]
[ 25 927]
[ 25 929]
[ 33 930]
[ 33 929]
[ 14 931]
[ 14 933]
[ 20 929]
[ 20 930]
[ 24 924]
[ 24 927]
[ 29 929]
[ 29 929]
[ 25 932]
[ 25 933]
[ 46 928]
[ 46 928]
[ 12 930]
[ 12 928]
[ 24 926]
[ 24 926]
[ 24 925]
[ 24 923]
[ 14 923]
[ 14 925]
[ 17 925]
[ 17 924]
[ 13 924]
[ 13 925]
[  8 921]
[  8 920]
[ 23 924]
[ 23 926]
[ 22 923]
[ 22 925]
[ 26 929]
[ 26 928]
[ 28 934]
[ 28 933]
[ 67 932]
[ 67 937]
[ 26 930]
[ 26 929]
[ 24 926]
[ 24 926]
[ 11 924]
[ 11 920]
[  8 923]
[  8 927]
[ 13 920]
[ 13 923]
[ 24 920]
[ 24 922]
[  8 922]
[ 16 928]
[ 13 401]
[ 37 401]
[ 37 403]


In [56]:
import json

def json2jsonl(df, writename):
    with open(writename, 'w', encoding='utf8') as f:
        for i, j in df.to_records(index=False):
            d = dict()
            d['src'] = j.strip()
            d['trg'] = i.strip()
            json.dump(d, f)
            f.write('\n')

In [57]:
json2jsonl(prepared_pauq_train, "train.jsonl")

In [58]:
prepared_pauq_train

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
...,...,...
8795,"SELECT t2.house_number , t1.name FROM LOCATI...",where can i find a restaurant in the bay area ...
8796,"SELECT t2.house_number , t1.name FROM LOCATI...",give me a good restaurant in the bay area ? ge...
8797,"SELECT t2.house_number , t1.name FROM LOCATI...",what are some good restaurants in the bay area...
8798,"SELECT t2.house_number , t1.name FROM LOCATI...",give me some good restaurants in the bay area ...
